    This file is a part of the lor_neuro_rat project.
    Copyright (C) 2019 anonimous

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

    Please contact with me by E-mail: shkolnick.kun@gmail.com

# Подготовка датасета для бинарного классификатора
Часть 2

In [ ]:
import pandas as pd

In [ ]:
#Загрузка
msg_of_interest = pd.read_pickle('data/MsgOfInterestDataFrame.pkl')
msg_of_interest.head()

In [ ]:
reason_lut      = pd.read_csv('data/markup/reasons_lut.csv', sep = ',')
print(reason_lut.head())
reason_lut = dict(zip(list(reason_lut['Reason'].values), list(reason_lut['Class1'].values)))
print(reason_lut[' ЯННП (-7)'])

In [ ]:
class_lut       = pd.read_csv('data/markup/class_lut.csv', sep = ',')
print(class_lut.head())

class_lut = dict(zip(list(class_lut['Class'].values), list(class_lut['Use'].values)))
print(class_lut[15])

In [ ]:
import re
msg_of_interest['Class'] = msg_of_interest['Reason'].apply(lambda x: reason_lut[re.sub(r',',';',x)])
msg_of_interest['Use']   = msg_of_interest['Class'].apply(lambda x: class_lut[x])
msg_of_interest.tail()

In [ ]:
positive = msg_of_interest[msg_of_interest['Use'] == 1].copy().reset_index(inplace=False, drop=True)
positive.to_pickle('data/Positive.pkl')

In [ ]:
positive.describe()


In [ ]:
positive.head()

In [ ]:
topics_of_interest = []

for t in list(positive['TopId'].values):
    if t not in topics_of_interest:
        topics_of_interest.append(t)

print('Промодерировано топиков: ', len(topics_of_interest))

In [ ]:
df = pd.read_pickle('data/all_data_as_dataframe.pkl')
messages_of_inerest = df[df['TopId'].apply(lambda x: x in topics_of_interest)]

In [ ]:
print(len(df))
print(len(messages_of_inerest))

In [ ]:
negative = messages_of_inerest[messages_of_inerest['DelReason'] == '']
#Интересуют только комментарии
negative = negative[negative['MsgId'].apply(lambda x: 'topic-' not in x)]
negative.head()

In [ ]:
print(len(negative))

In [ ]:
negative = negative.copy().reset_index(inplace=False, drop=True)
negative['Score'] = [0]*len(negative)
negative['Class'] = [0]*len(negative)
negative['Use']   = [0]*len(negative)
negative.to_pickle('data/Negative.pkl')

In [ ]:
negative.head()

In [ ]:
from sklearn.utils import shuffle
negative = shuffle(negative)

In [ ]:
dataset = pd.concat([positive, negative[0:len(positive)*1]], ignore_index=True, copy=True, sort=False)
#dataset = pd.concat([positive, negative], ignore_index=True, copy=True, sort=False)
dataset.describe()

In [ ]:
dataset.to_pickle('data/Dataset1.pkl')